In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import nibabel as nb
import SimpleITK as sitk


import HeadCT_motion_correction_PAR.functions_collection as ff
import HeadCT_motion_correction_PAR.Data_processing as dp
import HeadCT_motion_correction_PAR.motion_simulator.motion_simulation.ct_basic as basic
import HeadCT_motion_correction_PAR.motion_simulator.transformation as transform

import CTProjector.src.ct_projector.projector.numpy as ct_projector
import CTProjector.src.ct_projector.projector.numpy.fan_equiangluar as ct_fan
import CMR_HFpEF_Analysis.motion_correction.Bspline as Bspline
main_path = '/mnt/mount_zc_NAS/head_phantom_raw/processed'
our_projection = nb.load(os.path.join('/mnt/mount_zc_NAS/motion_correction/data/simulated_data_3D_spline/', 'MO101701M000001/MO001A000001/random_1/projection.nii.gz'))

# backproject the sinogram:

In [23]:
input_dir = os.path.join(main_path, 'study_4/scan6/processed_0/prjs_fan')
projector = ct_projector.ct_projector()
projector.from_file('./projector_fan_scanner.cfg')
prjs, projector = ff.read_projection_data(
        input_dir, projector, 0, -1, 1, 4
    )

prjs_hann = ff.apply_hann(prjs, projector)
# for k in vars(projector):
#         print (k, '=', getattr(projector, k))

Reading data from 27 files
0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,prjs shape  (1, 27, 1440, 4, 4, 448)
prjs shape  (27, 1440, 4, 448)


In [24]:
# # save
# prjs_save_folder = os.path.join(os.path.dirname(input_dir), 'prjs_hann')
# ff.make_folder([prjs_save_folder])
# print(prjs_hann.shape)
# # nb.save(nb.Nifti1Image(prjs_hann, our_projection.affine), os.path.join(prjs_save_folder,'projection_original.nii.gz'))
# nb.save(nb.Nifti1Image(prjs_hann, our_projection.affine), os.path.join(prjs_save_folder,'projection_averaged.nii.gz'))

(27, 1440, 4, 448)


In [19]:
# #  if nz_slcie = 1, then average every 4 slices
# prjs_reshape = np.transpose(prjs_hann,[0,2,1,3])
# prjs_reshape = np.reshape(prjs_reshape, [prjs_reshape.shape[0] * prjs_reshape.shape[1], prjs_reshape.shape[2], prjs_reshape.shape[3]])
 
# prjs_average = np.zeros([prjs_reshape.shape[0] // 4, prjs_reshape.shape[1], prjs_reshape.shape[2]])
# for i in range(0,prjs_reshape.shape[0] // 4):
#     prjs_average[i,...] = np.mean(prjs_reshape[(4 * i) : (4 * i + 4),...], axis = 0)
# print(prjs_average.shape)

# prjs_hann = np.reshape(prjs_average, [27, 4, 1440, 448])
# prjs_hann = np.transpose(prjs_hann, [0,2,1,3])

# projector = ct_projector.ct_projector()
# projector.from_file('./projector_fan_scanner.cfg')
# projector.nv = 4
# projector.nz = 4
# projector.dv = projector.dv * 4
# projector.dz = projector.dv

(108, 1440, 448)


In [18]:
angles = projector.get_angles()
prjs_hann_C = np.copy(prjs_hann, 'C')
fprjs_hann = ct_fan.ramp_filter(projector, prjs_hann_C, 'rl')
recon = ct_fan.fbp_bp(projector, fprjs_hann, angles)

# angles = projector.get_angles()
# prjs = np.copy(prjs, 'C')
# fprjs = ct_fan.ramp_filter(projector, prjs, 'hann')
# recon = ct_fan.fbp_bp(projector, fprjs, angles)


recon = recon / 0.0193 * 1000 - 1000
recon_final_original = np.concatenate(recon, axis = 0)
recon_final_original = recon_final_original.astype(np.int16)

# sitk_recon = sitk.GetImageFromArray(recon_final_original)
# sitk_recon.SetSpacing([float(projector.dx), float(projector.dy), float(projector.dz)])
# output_dir = os.path.join(os.path.dirname(input_dir), 'recon')
# ff.make_folder([output_dir])
# sitk.WriteImage(sitk_recon, os.path.join(output_dir, 'recon22.nii.gz'))

# Simulate motion

positive move in signogram theta direction: sinogram down, head rotation clock-wise in z, positive degree if using transformation

positive move in z direction: head move up, so for the same indexed slice, moved head shows the plane closer to the mouth compared to original head, negative tz if using transformation

change from study 4 scan 3 to study 4 scan 6 is tx negative move if using transformation

In [2]:
# all studies:
# initial state
prjs4 = nb.load(os.path.join(main_path, 'study_4/scan4/processed_0/prjs_hann/projection_original.nii.gz')).get_fdata()
prjs4_reshape = np.transpose(prjs4,[0,2,1,3])
prjs4_reshape = np.reshape(prjs4_reshape, [prjs4_reshape.shape[0] * prjs4_reshape.shape[1], prjs4_reshape.shape[2], prjs4_reshape.shape[3]])

prjs3 = nb.load(os.path.join(main_path, 'study_4/scan3/processed_0/prjs_hann/projection_original.nii.gz')).get_fdata()
prjs3_reshape = np.transpose(prjs3,[0,2,1,3])
prjs3_reshape = np.reshape(prjs3_reshape, [prjs3_reshape.shape[0] * prjs3_reshape.shape[1], prjs3_reshape.shape[2], prjs3_reshape.shape[3]])

prjs5 = nb.load(os.path.join(main_path, 'study_4/scan5/processed_0/prjs_hann/projection_original.nii.gz')).get_fdata()
prjs5_reshape = np.transpose(prjs5,[0,2,1,3])
prjs5_reshape = np.reshape(prjs5_reshape, [prjs5_reshape.shape[0] * prjs5_reshape.shape[1], prjs5_reshape.shape[2], prjs5_reshape.shape[3]])

prjs6 = nb.load(os.path.join(main_path, 'study_4/scan6/processed_0/prjs_hann/projection_original.nii.gz')).get_fdata()
prjs6_reshape = np.transpose(prjs6,[0,2,1,3])
prjs6_reshape = np.reshape(prjs6_reshape, [prjs6_reshape.shape[0] * prjs6_reshape.shape[1], prjs6_reshape.shape[2], prjs6_reshape.shape[3]])

print(prjs4.shape)


In [83]:
# define motions
tz_list = np.array([0.   , -0.   , -0.707   , -0.707, -0.707, 
              -1.414, -1.414, -1.414, -2.828, -2.828,
              -2.828, -2.828, -3.535, -3.535, -3.535, 
              -3.535,-3.535, -3.535, -2.828,-2.828,
              -2.828, -2.828, -3.535, -3.535, -3.535 ]) *-1

# tx_list = np.array([0,0,0,0, 0, 
#               0,  0,   0,   -2, -2, 
#               -2, -2, -2, -2, -2,
#               -2, -4, -4, -4, -4,
#               -4, -4, -4, -4, -4 ])
tx_list = [0] * 25
ty_list = [0] * 25

rz_list = np.array([0. ,0, 0.25, 0.50, 0.75, 
              0.75, 0.75, 1, 1, 1,
              1, 1.25, 1.50, 1.75, 2.0,
              2.50, 2.50, 2.50, 2.75, 3.0,
              3.50, 3.50, 3.25, 3.00, 2.75]) *-1
rx_list = [0] * 25
ry_list = [0] * 25

amplitude_tz_mm = Bspline.control_points(np.linspace(1,25,25), np.asarray(tz_list), 5 )
amplitude_tx_mm = Bspline.control_points(np.linspace(1,25,25), np.asarray(tx_list) * 0.601, 5 )
amplitude_ty_mm = Bspline.control_points(np.linspace(1,25,25), np.asarray(ty_list), 5 )
amplitude_rz_degree = Bspline.control_points(np.linspace(1,25,25), np.asarray(rz_list), 5 )
amplitude_rx_degree = Bspline.control_points(np.linspace(1,25,25), np.asarray(rx_list), 5 )
amplitude_ry_degree = Bspline.control_points(np.linspace(1,25,25), np.asarray(ry_list), 5 )
print(amplitude_tz_mm, amplitude_tx_mm, amplitude_ty_mm, amplitude_rz_degree, amplitude_rx_degree, amplitude_ry_degree)

[ 0.    -1.414 -3.535 -4.242 -3.535] [0. 0. 0. 0. 0.] [0. 0. 0. 0. 0.] [ 0.   -1.   -1.5  -2.75 -3.75] [0. 0. 0. 0. 0.] [0. 0. 0. 0. 0.]


In [84]:
# divide sinogram in to 25 segments (56 * 25 = 1400 + 40 slices same as the last segment) and simulate motions
increment = 1400 // 25
prjs_new = np.zeros_like(prjs4_reshape)

for i in range( 0, 26):

    if i == 25:
        tz = tz_list[-1]
        rz = rz_list[-1]
        tx = tx_list[-1]
    else:
        tz = tz_list[i]
        rz = rz_list[i]
        tx = tx_list[i]

    if i < 25:
        slice_index = [increment * i, increment * (i+1)]
    else:
        slice_index = [increment * 25 , prjs4_reshape.shape[1]]
    
    if tx == 0:
        p = np.copy(prjs4_reshape)
        use = 4
    elif tx == 1 or tx == 2:
        p = np.copy(prjs3_reshape)
        use = 3
    elif tx == -2:
        p = np.copy(prjs5_reshape)
        use = 5
    elif tx == -4:
        p = np.copy(prjs6_reshape)
        use = 6

    p = dp.move_3Dimage(p, [int(np.round(-tz / 0.707)), int(np.round(rz/360 * 1440)),0])
    print(i, slice_index, tx, 'sinogram use: ', use, -tz, int(np.round(-tz / 0.707)), rz, int(np.round(rz/360 * 1440)))

    prjs_new[:, slice_index[0]: slice_index[1], :] = p[:, slice_index[0]: slice_index[1], :]

0 [0, 56] 0 sinogram use:  4 -0.0 0 -0.0 0
1 [56, 112] 0 sinogram use:  4 0.0 0 -0.0 0
2 [112, 168] 0 sinogram use:  4 0.707 1 -0.25 -1
3 [168, 224] 0 sinogram use:  4 0.707 1 -0.5 -2
4 [224, 280] 0 sinogram use:  4 0.707 1 -0.75 -3
5 [280, 336] 0 sinogram use:  4 1.414 2 -0.75 -3
6 [336, 392] 0 sinogram use:  4 1.414 2 -1.0 -4
7 [392, 448] 0 sinogram use:  4 1.414 2 -1.0 -4
8 [448, 504] 0 sinogram use:  4 2.828 4 -1.0 -4
9 [504, 560] 0 sinogram use:  4 2.828 4 -1.0 -4
10 [560, 616] 0 sinogram use:  4 2.828 4 -1.0 -4
11 [616, 672] 0 sinogram use:  4 2.828 4 -1.25 -5
12 [672, 728] 0 sinogram use:  4 3.535 5 -1.5 -6
13 [728, 784] 0 sinogram use:  4 3.535 5 -1.75 -7
14 [784, 840] 0 sinogram use:  4 3.535 5 -2.0 -8
15 [840, 896] 0 sinogram use:  4 3.535 5 -2.5 -10
16 [896, 952] 0 sinogram use:  4 4.242 6 -2.5 -10
17 [952, 1008] 0 sinogram use:  4 4.242 6 -2.5 -10
18 [1008, 1064] 0 sinogram use:  4 4.242 6 -2.75 -11
19 [1064, 1120] 0 sinogram use:  4 4.242 6 -3.0 -12
20 [1120, 1176] 0 sinog

In [85]:
# save sinogram with original z resolution 
study = 'study_19'
main_save_folder = os.path.join('/mnt/mount_zc_NAS/motion_correction/data/phantom_data/', study)
ff.make_folder([main_save_folder])
nb.save(nb.Nifti1Image(prjs_new, our_projection.affine), os.path.join(main_save_folder,'projection_HR.nii.gz'))

In [86]:
# reconstruct images using z resolution = 2.828mm
prjs_new_average = np.zeros([prjs_new.shape[0] // 4, prjs_new.shape[1], prjs_new.shape[2]])
for i in range(0,prjs_new.shape[0] // 4):
    prjs_new_average[i,...] = np.mean(prjs_new[(4 * i) : (4 * i + 4),...], axis = 0)

print(prjs_new_average.shape)
prjs_new_average = np.reshape(prjs_new_average, [prjs_new_average.shape[0] // 4, 4, prjs_new_average.shape[1],prjs_new_average.shape[2]])
prjs_new_average = np.transpose(prjs_new_average, [0,2,1,3])
print(prjs_new_average.shape)

projector = ct_projector.ct_projector()
projector.from_file('./projector_fan_scanner.cfg')
projector.nv = 4
projector.nz = 4
projector.dv = projector.dv * 4
projector.dz = projector.dv

angles = projector.get_angles()
prjs_new_averageC = np.copy(prjs_new_average, 'C')
fprjs = ct_fan.ramp_filter(projector, prjs_new_averageC, 'rl')
recon = ct_fan.fbp_bp(projector, fprjs, angles)
recon = recon / 0.0193 * 1000 - 1000

recon_final = np.concatenate(recon, axis = 0)
recon_final = recon_final.astype(np.int16)
sitk_recon = sitk.GetImageFromArray(recon_final)
sitk_recon.SetSpacing([float(projector.dx), float(projector.dy), float(projector.dz)])

output_dir = os.path.join(main_save_folder, 'simulated_imgs_raw')
ff.make_folder([output_dir])
sitk.WriteImage(sitk_recon, os.path.join(output_dir, 'recon_motion.nii.gz'))


# save partial
recon_final_partial = np.concatenate(recon, axis = 0)[45:95,...]
sitk_recon = sitk.GetImageFromArray(recon_final_partial)
sitk_recon.SetSpacing([float(projector.dx), float(projector.dy), float(projector.dz)])
sitk.WriteImage(sitk_recon, os.path.join(output_dir, 'recon_motion_partial.nii.gz'))


# save motion
parameter_file = os.path.join(main_save_folder,'motion_parameters.npy')
np.save(parameter_file, np.array([[amplitude_tx_mm],[amplitude_ty_mm],[amplitude_tz_mm],[amplitude_rx_degree],[amplitude_ry_degree],[amplitude_rz_degree],[0], [0], [1440], [500]], dtype=object))

a = np.stack([np.asarray(tx_list) * 0.601, np.asarray(ty_list), np.asarray(tz_list), np.asarray(rx_list), np.asarray(ry_list), np.asarray(rz_list)])
np.save(os.path.join(main_save_folder,'motion_each_point.npy'), a)

# save sinogram
prjs_new_reshape = np.reshape(np.transpose(prjs_new_average,[0,2,1,3]), [prjs_new_average.shape[0] * prjs_new_average.shape[2] , prjs_new_average.shape[1], prjs_new_average.shape[-1]])
print(prjs_new_reshape.shape)
nb.save(nb.Nifti1Image(prjs_new_reshape, our_projection.affine), os.path.join(main_save_folder,'projection.nii.gz'))

(108, 1440, 448)
(27, 1440, 4, 448)
(108, 1440, 448)
